In [48]:
import numpy as np
import scipy.io as spio
from os import listdir
from os.path import isfile, join
import timeit as tt
import json
import sys
#import pandas
np.set_printoptions(threshold=sys.maxsize)

In [49]:
test_config={
    "matfile":"",
    "comp_base":"",
    "midFreq":"",
    "inband":"",
    "bandnum":"",
    "lbl_len":""
}

In [50]:
def write_speech_to_mat(source, destination):
    pass

In [51]:
def load_file(file_name):
    pass

In [52]:
def load_matrix_weight(mat_file):
    pass

In [53]:
def sigmoid(X):
    return 1/(1+np.exp(-X))

def relu(X):
    return np.maximum(0,X)

def softmax(X):
    expo = np.exp(X)
    expo_sum = np.sum(np.exp(X))
    return expo/expo_sum

In [54]:
def getFilesInFloder(folderPath):
    onlyfiles = [f for f in listdir(folderPath) if isfile(join(folderPath, f))]
    return onlyfiles

In [55]:
def genICFilterMatrix(bandnum, y_length, mid_freq_matrix):
    ret_icfilter = np.zeros((bandnum, y_length),dtype=float)
    for i in range(bandnum):
        if i == 9:
            for j in range(y_length):
                ret_icfilter[i][j]=10**((-360*abs(np.log10(15.625*(j-1)+1)-np.log10(15.625*(mid_freq_matrix[i]-1))))/20)
        elif i == 10:
            for j in range(y_length):
                ret_icfilter[i][j]=10**((-360*abs(np.log10(15.625*(j-1)+1)-np.log10(15.625*(mid_freq_matrix[i]-1))))/20)
        else:
            for j in range(y_length):
                ret_icfilter[i][j]=10**((-20*12*abs(np.log10(15.625*(j-1)+1)-np.log10(15.625*(mid_freq_matrix[i]-1))))/20)
    
    return ret_icfilter

In [56]:
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
            np.int16, np.int32, np.int64, np.uint8,
            np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32, 
            np.float64)):
            return float(obj)
        elif isinstance(obj,(np.ndarray,)): #### This is the fix
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [57]:
# rec_structure = 
# {
#     "test_global_parameters":
#     {
#         "start_frame"=,
#         "end_frame"=,
#         "step_quantity"=,
#     }
#     ,
#     "test_record":
#     {
#         "":""
        
#     }
    

# }

In [58]:
def runAccuracyTest(weights_file=None, test_matfile=None, comp_base=1, threshold=-145, lbl_len=513,
                    icfilterary=None ,startPoint=1600, stepQuantity=400 \
                   ):
    """doing the accuracy of input matfile
    
    parameters:on-default argument fo
    test_matfile: to test mat file path
    camp_base: accuracy co
    
    """
    weights = spio.loadmat(weights_file)
    w1 = weights["w1"]
    w2 = weights["w2"]
    w3 = weights["w3"]
    b1 = weights["b1"]
    b2 = weights["b2"]
    b3 = weights["b3"]
    mat_data = spio.loadmat(test_matfile)
    
    inband = 4
    bandnum = 10
    count_answer_1 = 0
    count_answer_0 = 0
    count_test_1 = 0
    count_test_0 = 0

    y_answer = np.empty([0,0],dtype=int)
    y_test = np.empty([0,0])

    local_power = np.empty([0,0])
    local_power_normalize = np.empty([0,0],dtype=int)
    x_all = np.empty([0,0])
    x_normalize_all = np.empty([0,0])
    
    s_len = int(400*(np.floor(len(mat_data['s'])/400)))
    sample_num = inband * bandnum
    test_counter = 0
#     loop_record={}
#     loop_record["mat_file"]=test_matfile
#     loop_record["comp_base"]=comp_base
#     constant_str1 = "loop"
    for i in range(startPoint,s_len,stepQuantity):
        test_counter += 1
        #divide four segment every 25ms
        idx1 = i-1600
        idx2 = i-1200
        idx3 = i-800
        idx4 = i-400
        s1 = mat_data['s'][idx1:idx2] #0-399
        s2 = mat_data['s'][idx2:idx3] #400-799run_test_main,number
        s3 = mat_data['s'][idx3:idx4] #800-1199
        s4 = mat_data['s'][idx4:i]    #1200-1599
#         loop_record={
#             "s1":"{}_{}".format(idx1,idx2),
#             "s2":"{}_{}".format(idx2,idx3),
#             "s3":"{}_{}".format(idx3,idx4),
#             "s4":"{}_{}".format(idx4,i)
#         }
#         loop_record["threshold"]=threshold      
#         s1 = mat_data['s'][i-1600:i-1200]#0-399
#         s2 = mat_data['s'][i-1200:i-800] #400-799
#         s3 = mat_data['s'][i-800:i-400]  #800-1199
#         s4 = mat_data['s'][i-400:i]      #1200-1599

        s_fft_1 = np.fft.fft(s1,n=1024,axis=0) 
        s_fft_2 = np.fft.fft(s2,n=1024,axis=0) 
        s_fft_3 = np.fft.fft(s3,n=1024,axis=0) 
        s_fft_4 = np.fft.fft(s4,n=1024,axis=0) 

        s_fft_1_513 = np.split(s_fft_1,[0,lbl_len-1],axis=0)[1]
        s_fft_2_513 = np.split(s_fft_2,[0,lbl_len-1],axis=0)[1]
        s_fft_3_513 = np.split(s_fft_3,[0,lbl_len-1],axis=0)[1]
        s_fft_4_513 = np.split(s_fft_4,[0,lbl_len-1],axis=0)[1]

        # there is one error element in every array in position 9 of each array
        x1 = np.log(np.matmul(icfilterary,(np.abs(s_fft_1_513)**2)) + 0.0001)
        x2 = np.log(np.matmul(icfilterary,(np.abs(s_fft_2_513)**2)) + 0.0001)
        x3 = np.log(np.matmul(icfilterary,(np.abs(s_fft_3_513)**2)) + 0.0001)
        x4 = np.log(np.matmul(icfilterary,(np.abs(s_fft_4_513)**2)) + 0.0001)

        x= np.array([x1,x2,x3,x4]).reshape(1,sample_num)
        x_40 = x[0:40]
        
        max_ele = np.amax(x_40,axis=1)
        min_ele = np.amin(x_40,axis=1)
        sum_of_x = np.sum(x_40)
        
        # normalize
        x_normalize = (x_40-min_ele)/(max_ele-min_ele+0.0001)

        #counting the test
        if sum_of_x > threshold:
            #performing the model weight mulplications
            answer = softmax(np.matmul(relu(np.matmul(relu(np.matmul(x_normalize,w1)+b1),w2)+b2),w3)+b3)
            if answer[0,0] > answer[0,1]:
                y_test = np.append(y_test, 0)
                count_test_0 += 1
            else:
                y_test = np.append(y_test, 1)
                count_test_1 += 1
        else:
            y_test = np.append(y_test, 0)
            count_test_0 += 1

        #counting our answer
        if comp_base == 1:
            if sum_of_x > threshold:
                y_answer = np.append(y_answer,1)
                count_answer_1 += 1
            else:
                y_answer = np.append(y_answer,0)
                count_answer_0 += 1
        else:
            y_answer = np.append(y_answer,0)
            count_answer_0 += 1
            
#     loop_record["total_loop"] = test_counter
    acc = 0
    LED = 0
    wrong = np.empty([0,0])
#     loop_record["y_answer"]=y_answer.tolist()
    y_answer_len = len(y_answer)
#     loop_record["y_answer_len"]=y_answer_len
    
#     loop_record["y_test"] = y_test.tolist()
    y_test_len = len(y_test)
#     loop_record["y_test_len"] = y_test_len
    
    if comp_base == 1: #if our test data is human voice
        for c in range(y_answer_len):
            acc += abs(y_answer[c]-y_test[c])

        acc = (y_answer_len-acc)/y_answer_len
    else:
        acc = count_test_0/(count_test_0+count_test_1)
    
#     loop_record["accuracy"]=acc
#     return loop_record
    print("acc is ", acc)
    print("{} test completed\n".format(test_matfile))
    return acc #json.dumps(loop_record,cls=NumpyEncoder)


In [43]:
middleFreq = [16, 20, 26, 36, 48, 60, 80, 101, 256, 353]
_weights = "../../Weights/w_original_128x32.mat
y_len = 513
icfilterary = genICFilterMatrix(10, y_len, middleFreq)[:,1:]
def run_multitest_main():
    _matfilespath = "../../speechData/forKriz/clean_mat/"
    cflag = 1
    filelist = getFilesInFloder(_matfilespath)
    print("files in _matfilespath are {}".format(filelist))
    acclist = []
    for f in filelist:
        _acc = runAccuracyTest(_weights, _matfilespath+f, icfilterary=icfilterary,comp_base=cflag)
        acclist.append(_acc)
#         with open("logfiles/noise1016_M1_1/"+f+".log", 'w') as outfile:
#             outfile.write(str(log))
#             outfile.flush()
    #calculating average of all accs
    avg = sum(acclist)/len(acclist)
    print("The average is ", round(avg,5))

/home/user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in log10
  if sys.path[0] == '':
/home/user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in log10
  


In [44]:
#perform performance measurement
print("total test time is  : {}".format(tt.timeit(run_multitest_main,number=1)))

files in _matfilespath are ['ABJones_5_lyrics.wav.mat', 'amy_15_lyrics.wav.mat', 'amy_14_lyrics.wav.mat', 'amy_3_lyrics.wav.mat', 'amy_11_lyrics.wav.mat', 'amy_5_lyrics.wav.mat', 'amy_2_lyrics.wav.mat', 'amy_10_lyrics.wav.mat', 'amy_6_lyrics.wav.mat', 'amy_12_lyrics.wav.mat', 'ABJones_1_lyrics.wav.mat', 'ABJones_4_lyrics.wav.mat', 'amy_13_lyrics.wav.mat', 'amy_1_lyrics.wav.mat', 'amy_9_lyrics.wav.mat', 'ABJones_3_lyrics.wav.mat', 'ABJones_2_lyrics.wav.mat', 'amy_4_lyrics.wav.mat', 'amy_7_lyrics.wav.mat', 'amy_8_lyrics.wav.mat']
acc is  1.0
../../speechData/forKriz/clean_mat/ABJones_5_lyrics.wav.mat test completed

acc is  1.0
../../speechData/forKriz/clean_mat/amy_15_lyrics.wav.mat test completed



/home/user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: overflow encountered in exp
  
/home/user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in exp
  if __name__ == '__main__':
/home/user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


acc is  0.9974747474747475
../../speechData/forKriz/clean_mat/amy_14_lyrics.wav.mat test completed

acc is  1.0
../../speechData/forKriz/clean_mat/amy_3_lyrics.wav.mat test completed

acc is  0.9974747474747475
../../speechData/forKriz/clean_mat/amy_11_lyrics.wav.mat test completed

acc is  1.0
../../speechData/forKriz/clean_mat/amy_5_lyrics.wav.mat test completed

acc is  1.0
../../speechData/forKriz/clean_mat/amy_2_lyrics.wav.mat test completed

acc is  0.9949494949494949
../../speechData/forKriz/clean_mat/amy_10_lyrics.wav.mat test completed

acc is  1.0
../../speechData/forKriz/clean_mat/amy_6_lyrics.wav.mat test completed

acc is  1.0
../../speechData/forKriz/clean_mat/amy_12_lyrics.wav.mat test completed

acc is  1.0
../../speechData/forKriz/clean_mat/ABJones_1_lyrics.wav.mat test completed

acc is  1.0
../../speechData/forKriz/clean_mat/ABJones_4_lyrics.wav.mat test completed

acc is  0.9974747474747475
../../speechData/forKriz/clean_mat/amy_13_lyrics.wav.mat test completed

acc

In [34]:
# def single_file_test():
#     _weights = "../weight/8+2band(25ms)/20200110/weight_512x32_20200110_15786815140978317.mat"
#     y_len = 513
#     middleFreq = [16, 20, 26, 36, 48, 60, 80, 101, 256, 353]
#     icfilterary = genICFilterMatrix(10, y_len, middleFreq)[:,1:]
#     _matfile = "../data/noise_mat/Speakers_Train/bear.dat.mat"
#     _matfile2 = "../data/test_mat/cow.dat.mat"
#     testlst = []
#     _acc, jsonlog = runAccuracyTest(_weights,_matfile2,icfilterary=icfilterary)
    
#     with open("test_singlefile.log","w") as f:
#         f.write(str(jsonlog))
#         f.flush()
#     # print("_acc is {}".format(_acc))
#     print(jsonlog)

In [19]:
# print("total test time is  : {}".format(tt.timeit(single_file_test,number=1)))

In [12]:
#load mat file
# weights = spio.loadmat('../weight/8+2band(25ms)/20200110/weight_512x32_20200110_15786815140978317.mat')
# w1 = weights["w1"]
# w2 = weights["w2"]
# w3 = weights["w3"]
# b1 = weights["b1"]
# b2 = weights["b2"]
# b3 = weights["b3"]
s = spio.loadmat('../data/noise_mat/Speakers_Train/bear.dat.mat')
target = 0 # 0: mean we load the noise data; 1: load the human speech
# print("s length is {}".format(len(s)))


In [13]:
print(getFilesInFloder("../data/noise_mat/A1/"))

['A1_P20_ForestDay.dat.mat', 'A1_上升.dat.mat', 'A1_驢.dat.mat', 'A1_開場.dat.mat', 'A1_Holmes坐皮椅.dat.mat', 'A1_Kz踏office地毯.dat.mat', 'A1_手持officeMic敲桌子 .dat.mat', 'A1_P20_Sailing.dat.mat', 'A1_十指關節敲桌子 .dat.mat', 'A1_拍軟墊牆壁.dat.mat', 'A1_P20_Flow.dat.mat', 'A1_P20_Impromuto.dat.mat', 'A1_環境音.dat.mat', 'A1_敲白板.dat.mat']


In [14]:
# midfre = [16, 20, 26, 36, 48, 60, 80, 101, 256, 353]
# second_range = 512
# icfilter = np.zeros((bandnum, second_range),dtype=float)

In [103]:
tmp_ary = np.array([1,1])
for i in range(3):
    tmp_ary2 = np.random.randint(low = 1, high = 99, size = 513)
    tmp_ary = np.append(tmp_ary,tmp_ary2)
tmp_ary = tmp_ary.reshape(len(tmp_ary),1)
print(tmp_ary.shape)

(1541, 1)
